### This script connects to a MongoDB database, downloads stock data, preprocesses it, and uploads it to the database.

Steps:

1. Read in CSV file 'Complete-List-of-Listed-Companies-on-Singapore-Stock-Exchange-052923.csv' to extract Trading Name and Code columns. These represent the stock symbols we're interested in.
2. Append '.SI' to the end of each Code. This is required because the yFinance API uses '.SI' to denote stocks listed on the Singapore Exchange.
3. Establish a connection to the MongoDB database using the connection string from 'config.env'.
4. Download stock data for each symbol from the yFinance API. If a symbol is delisted or unavailable, it is skipped and a warning message is printed.
5. Preprocess the downloaded data by converting it to a time-series format and setting appropriate column names.
6. Upload the preprocessed data to the MongoDB database. Each symbol's data is uploaded as a separate document in the 'ticker_data' collection.

In [ ]:
# Define the stock symbols and time range
# symbols = ['U96.SI', 'NLC.SI']
# symbols_list = ['L5I.SI', '541.SI', 'QZG.SI']
symbols_list = ['40F.SI']
print(symbols_list)

['40F.SI']


In [ ]:
# Find the matching row in SGXdf for the current symbol
matched_row = SGXdf[SGXdf['Code'] == 'NLC.SI']

matched_row

,S.No.,Trading Name,Code,Sector
1,2,5E Resources,NLC.SI,Industrial & Commercial Services


In [ ]:
# Load the .env file
load_dotenv('../server/config.env')

# Get the connection string
connection_string = os.getenv('ATLAS_URI')

# Connect to MongoDB
client = pymongo.MongoClient(connection_string)
db = client['bullsai']
collection = db['ticker_data']

# Get the latest end date and transaction count in the collection
latest_record = db.ticker_data.find().sort("bucket_end_date", pymongo.DESCENDING).limit(1)

# Initialize latest_end_date and latest_transaction_count
if latest_record.count() > 0:
    latest_end_date = latest_record[0]["bucket_end_date"]
    latest_transaction_count = latest_record[0]["transaction_count"]
else:
    latest_end_date = "2000-01-01"  # If no data is available, use a default start date
    latest_transaction_count = 0

# Determine start_date and end_date based on the latest end date
start_date = '2000-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')

# Fetch data from yFinance for the list of symbols
for symbol in symbols_list:
    try:
        # Fetch data from yFinance for the current symbol
        data = yf.download(symbol, start=start_date, end=end_date)
        
        # Check if data is fetched successfully
        if not data.empty:
            # Reset index to make the Date column accessible
            data.reset_index(inplace=True)

            # Preprocess the data
            data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
            data['Date'] = data['Date'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d'))

            # Find the matching row in SGXdf for the current symbol
            matched_row_sgx = SGXdf[SGXdf['Code'] == symbol]
            print(matched_row_sgx)

            # Define trading_name and transactions
            trading_name = matched_row_sgx['Trading Name'].values[0] if not matched_row_sgx.empty else None
            transactions = data.to_dict('records')  # Removed the drop operation

            # Check if data is successfully fetched
            if not data.empty:
                matched_row = db.ticker_data.find_one({"symbol": symbol})
                if matched_row:
                    # Check if the bucket_end_date is up-to-date
                    if matched_row['bucket_end_date'] == data['Date'].max():
                        print(f"Skipping update for symbol {symbol}. bucket_end_date is up-to-date.")
                        continue
                        
                   # Update existing document
                    db.ticker_data.update_one(
                        {"symbol": symbol},
                        {"$set": {"transactions": transactions,
                                "transaction_count": len(transactions),
                                "bucket_end_date": end_date,
                                "trading_name": trading_name}}
                    )

                else:
                    # Insert new document
                    db.ticker_data.insert_one({
                        "trading_name": trading_name,
                        "symbol": symbol,
                        "transaction_count": len(data),
                        "bucket_start_date": data['Date'].min(),
                        "bucket_end_date": data['Date'].max(),
                        "transactions": transactions
                    })
        else:
            print(f"No data fetched for symbol {symbol}. Skipping update.")
    except Exception as e:
        print(f"Failed to fetch data for symbol {symbol}: {e}")

# Close the MongoDB connection
client.close()


/var/folders/v3/t0psjdhs29g8j3cghfdfjd9w0000gn/T/ipykernel_93870/2367871250.py:16: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  if latest_record.count() > 0:


[*********************100%%**********************]  1 of 1 completed
   S.No. Trading Name    Code       Sector
0      1      3Cnergy  502.SI  Real Estate
[*********************100%%**********************]  1 of 1 completed
   S.No.  Trading Name    Code                            Sector
1      2  5E Resources  NLC.SI  Industrial & Commercial Services
[*********************100%%**********************]  1 of 1 completed
   S.No. Trading Name    Code                Sector
2      3     8Telecom  AZG.SI  Technology Equipment
Skipping update for symbol AZG.SI. bucket_end_date is up-to-date.
[*********************100%%**********************]  1 of 1 completed
   S.No. Trading Name    Code            Sector
3      4           9R  1Y1.SI  Industrial Goods
Skipping update for symbol 1Y1.SI. bucket_end_date is up-to-date.
[*********************100%%**********************]  1 of 1 completed
   S.No. Trading Name    Code            Sector
4      5          ABR  533.SI  Food & Beverages
[**********


1 Failed download:
['L5I.SI']: Exception('%ticker%: No timezone found, symbol may be delisted')



No data fetched for symbol L5I.SI. Skipping update.
[*********************100%%**********************]  1 of 1 completed
   S.No.    Trading Name    Code                            Sector
6      7  Abundance Intl  541.SI  Industrial & Commercial Services
[*********************100%%**********************]  1 of 1 completed
   S.No. Trading Name    Code             Sector
7      8    Abundante  570.SI  Mineral Resources
Skipping update for symbol 570.SI. bucket_end_date is up-to-date.
[*********************100%%**********************]  1 of 1 completed
   S.No.   Trading Name    Code                Sector
8      9  Accrelist Ltd  QZG.SI  Technology Equipment
[*********************100%%**********************]  1 of 1 completed
   S.No.      Trading Name    Code                            Sector
9     10  Acesian Partners  5FW.SI  Industrial & Commercial Services
[*********************100%%**********************]  1 of 1 completed
    S.No. Trading Name    Code            Sector
10     11


1 Failed download:
['BVP.SI']: Exception('%ticker%: No data found, symbol may be delisted')


...


No data fetched for symbol BEV.SI. Skipping update.
[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['5UO.SI']: Exception('%ticker%: No timezone found, symbol may be delisted')



No data fetched for symbol 5UO.SI. Skipping update.
[*********************100%%**********************]  1 of 1 completed
     S.No. Trading Name    Code       Sector
565    566           TA  PA3.SI  Real Estate
[*********************100%%**********************]  1 of 1 completed
     S.No.      Trading Name    Code            Sector
566    567  Tai Sin Electric  500.SI  Industrial Goods
[*********************100%%**********************]  1 of 1 completed
     S.No.    Trading Name    Code     Sector
567    568  Taka Jewellery  42L.SI  Retailers
[*********************100%%**********************]  1 of 1 completed
     S.No. Trading Name    Code               Sector
568    569      TalkMed  5G3.SI  Healthcare Services
[*********************100%%**********************]  1 of 1 completed
     S.No.  Trading Name    Code             Sector
569    570  Tat Seng Pkg  T12.SI  Applied Resources
[*********************100%%**********************]  1 of 1 completed
     S.No. Trading Name    Code


1 Failed download:
['SO7.SI']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")



No data fetched for symbol SO7.SI. Skipping update.
[*********************100%%**********************]  1 of 1 completed
     S.No.      Trading Name    Code            Sector
648    649  YZJ Shipbldg SGD  BS6.SI  Industrial Goods
[*********************100%%**********************]  1 of 1 completed
     S.No.      Trading Name    Code            Sector
649    650  Zheneng Jinjiang  BWM.SI  Renewable Energy
[*********************100%%**********************]  1 of 1 completed
     S.No.     Trading Name    Code     Sector
650    651  Zhongmin Baihui  5SR.SI  Retailers
[*********************100%%**********************]  1 of 1 completed
     S.No.    Trading Name    Code            Sector
651    652  Zhongxin Fruit  5EG.SI  Food & Beverages
[*********************100%%**********************]  1 of 1 completed
     S.No. Trading Name    Code                         Sector
652    653   ZICO Hldgs  40W.SI  Banking & Investment Services
[*********************100%%**********************]  1 of